In [1]:
import json
import inspect
import keras
from importlib import import_module
from copy import deepcopy

Using TensorFlow backend.


In [2]:
toolbox = 'Keras'
excluded_classes = ['K']
layers = [layer for layer in dir(keras.layers) 
          if layer[0].isupper() and layer not in excluded_classes]

In [3]:
sections = []
keras_layers = []
for class_name in layers:
    module = eval('keras.layers.%s.__module__' % class_name)
    if module.startswith('keras.layers.'):
        section = module[len('keras.layers.'):]
        sections.append(section)
        keras_layers.append(tuple((class_name, section)))

In [4]:
c = 'MaxPooling1D'
module = eval('keras.layers.%s()' % c)
config = module.get_config()
ports = [{ 'name': port,
           'input': True,
           'output': False,
           'visible': True,
           'editable': True,
           'default': config[port], 
           'code': []
         } for port in config.keys()];

In [5]:
sections = set(sections)
categories = [{ 'name': section, 'nodes': [] } for section in list(sections)]
dictionary = { 'name': toolbox, 'categories': categories}

In [6]:
def keras2giraffe(name, section):
    module = eval('keras.layers.%s()' % c)
    inputs = module.get_config()
    
    ports = [{ 'name': port,
               'input': False,
               'output': False,
               'visible': True,
               'editable': True,
               'default': inputs[port], 
               'code': []
             } for port in inputs.keys()];
    ports.insert(0, {
        'name': 'INPUT',
        'input': True,
        'output': False,
        'visible': True,
        'editable': True, 
        'code': []
    })
    ports.append({
        'name': 'OUTPUT',
        'input': False,
        'output': True,
        'visible': True,
        'editable': True,
        'code': []
    })
    
    return { 
        'name': name,
        'category': [section],
        'toolbox': toolbox,
        'code': [],
        'web_url': 'https://keras.io/layers/%s/%s' % (section, name),
        'ports': ports
    }

In [7]:
for layer in keras_layers:
    node = keras2giraffe(layer[0], layer[1])
    [category for category in categories if category['name'] == layer[1]][0]['nodes'].append(node)

In [8]:
with open('keras_nodes.json', 'w') as outfile:
    json.dump({'toolboxes': [dictionary]}, outfile, sort_keys=False, indent=2)
    # json.dump({'toolboxes': toolboxes}, outfile, separators=(',', ':'))

In [11]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\timvmou\\GitHub'